In [1]:
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
import nltk
from nltk import sent_tokenize, word_tokenize
import pandas as pd
from io import BytesIO
import csv

from flair.tokenization import SegtokTokenizer
from flair.tokenization import SegtokSentenceSplitter
from tqdm import tqdm
from difflib import SequenceMatcher
import re
import pickle
import re

In [42]:
def pdf2token(pdf,loc):
    rsrcmgr = PDFResourceManager()
    retstr = BytesIO()
    codec = 'utf-8'
    laparams = LAParams(char_margin=30, line_margin=2, boxes_flow=1)
    device = TextConverter(rsrcmgr, retstr, codec='utf-8', laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    caching = True
    fp = open(pdf, 'rb')
    writer = csv.writer(open(loc, 'w', newline='', encoding="utf-8"))
    pagenumber = 0
    pagenos=set()
    for pagenumber, page in enumerate(PDFPage.get_pages(fp,pagenos, maxpages=0,caching=True, check_extractable=True)):
        #Process pages one by one into interpreter
        if pagenumber is not None:
            interpreter.process_page(page)
        pagenumber = pagenumber+1
        
    #Convert Interpreter data into string
    data = retstr.getvalue()
    data = data.decode("ascii", "ignore")
    #print(f' >>>>>>NORMALSTART<<<<<< {data} >>>>>>>>END<<<<<<<')
    #sent_text = sent_tokenize(data) # NLTK sentence tokenizer (poor results)
    splitter = SegtokSentenceSplitter() # Flair sentence spliter + word tokenizer
    sent_text = splitter.split(data)
    #print(f' >>>>>>START<<<<<< {sent_text} >>>>>>>>END<<<<<<<')
    for sent in sent_text:
        writer.writerow([sent.to_tokenized_string().lower()])
        

def to_plain_sent(tokens): # Convert tokenized(cleaned) words back to sentence
    
    sent = " ".join([t.text for t in tokens])
    return sent.rstrip()

def txttrain(train):
    entpair = []
    eg = []
    ent = []
    with open(train, 'r', encoding="utf-8") as f:
        for line in f: 
            text = line.rstrip('\n')
            defi = []
            defi = text.lower().split(',')
            defi[0] = defi[0].replace(' ','I')
            if len(defi)>=3:
                for w in range(1,len(defi)-1):
#                    print(str(w))
                    defi[1] = defi[1]+', '+defi[w+1]
            defi[1] = defi[1].strip('\n "()')
            tokeniser = SegtokTokenizer() # Flair sentence tokenizer
            sent_text = tokeniser.tokenize(defi[1])
            new = to_plain_sent(sent_text)
#            print(f'Tok: {new} Old: {defi[1]}')
            temp = new
            #defi[1] = clean(defi[1])
            #defi[1] = " ".join(defi[1].split())
            #temp = defi[1]
            defi[1] = defi[0]
            defi[0] = temp
            print('[ '+defi[0]+' , '+defi[1]+' ]')
            eg.append(defi[0])
            ent.append(defi[1])
        entpair = list(zip(eg,ent))
    return(entpair)
            

    
def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip("[] ")
    string = string.strip("[] ")
    match = []
    '''
    i=0
    x = 0
    while x!= -1:
      x = string.find(pattern, i)
      if x!= -1:           
        tup = (x, x+len(pattern))
        #print('Value: '+pattern+'Location:    '+str(tup)+'\n\n'+string)
        match.append(tup)
        i = x+len(pattern)
    #match.pop()
    return match, string
    '''
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        print(f'{string} //||// {pattern}')
        match_list.append(match_tup)
    return match_list, string
    
        
def sen2tag(s, match_list):
    tokeniser = SegtokTokenizer() # Flair sentence tokenizer
    s = s.strip("[] ")
    
    word_list = []
    first = 0    
    endstr = None
    end_s,blank_s,blank_tok,end_tok = None,None,None,None
    for itr,multiple in enumerate(match_list):
        e_start = multiple[0]
        e_end = multiple[1]
        e_name = multiple[2]
        
        print(f'>> {e_start} {e_end} {e_name} First: {first} - {e_start}')
        if e_start-first>1: # Atleast one word is present that is not an entity
            blank_s = s[first:e_start]  # Assign it to blank sentence to be tagged 'O'
        
        ent_s = s[e_start:e_end]    # Assign sentence to be tagged with entity
        
        if itr==len(match_list)-1:
            end_s = s[e_end:]
        
        # Tokenize both types of sentence
        if blank_s:
            blank_tok = tokeniser.tokenize(blank_s)
        if end_s:
            end_tok = tokeniser.tokenize(end_s)
        ent_tok = tokeniser.tokenize(ent_s)
        
        if blank_tok:   # If blank sentence is not None
            for word in blank_tok:
                word_list.append(f'{word.text} O')
        for num,word in enumerate(ent_tok):
            if num == 0:
                word_list.append(f'{word.text} B-{e_name}')  # Tag the first word as Begining of entity
            else:
                word_list.append(f'{word.text} I-{e_name}')  # Tag rest as Inside entity
        if end_tok:
            for word in end_tok:
                word_list.append(f'{word.text} O')
            
        first = e_end        
        blank_s,end_s = None,None    # Reset blank sentence tokens to avoid duplicacy 
    return word_list

# Redundent function replaced by sent2tok
def mark_sentence(s, match_list):
    '''
    Marks all the entities in the sentence as per the BIO scheme. 
    '''
    tokeniser = SegtokTokenizer() # Flair sentence tokenizer

    s = s.strip("[] ")
    #print(s)
    
    word_dict = []
    first = 0    
    itr = 0
    endstr = None

    for multiple in match_list:
        beg = False 
        #print('current itteration- '+str(multiple))
        start = multiple[0]
        end = multiple[1]
        e_type = multiple[2]
        temp_str = s[start:end]
        tmp_list = tokeniser.tokenize(temp_str) #word_tokenize(temp_str)
        if itr!=(len(match_list)-1):
            substring = s[first:end]
            #print('Itteration: '+str(itr)+str(substring))
        else:
            substring = s[first:end]
            endstr = s[end:]
            #print('Itteration: '+str(itr)+str(substring))
            
        print('First Identifier:    '+tmp_list[0].text) 
        tokens = tokeniser.tokenize(substring)
        for word in tokens:
            x = word.text.find(tmp_list[0].text)
            if x!=0:
                word_dict.append(word.text+' '+'O')
            if x==0 and beg is False:    
                word_dict.append(word.text+' '+'B-' + e_type)
                beg = True
                print('Begiening detected : '+ word.text)
                continue
                
            if len(tmp_list) > 1 and beg is True:
                #print('We in the long sent: :: : : : : : : ')
                #print(f'Rest KEY = {str(tmp_list[1:])}')
                for w in tmp_list[1:]:
                    t = word.text.find(w.text)
                    if t==0:
                        #print(str(t)+' Inner detected : '+word+' idf> '+w)
                        word_dict.pop()
                        word_dict.append(word.text+' '+'I-' + e_type)
                        
        # Seperate non entities left out for final sentence to be marked 'O'
        if endstr is not None:
            tokens = tokeniser.tokenize(endstr)
            for word in tokens:
                word_dict.append(word.text+' '+'O')
        first=end
        itr=itr+1
    #print('Final Word outcome:::    '+str(word_dict)+'\n\n\n')
    return word_dict

def clean(text):
    '''
    Just a helper fuction to add a space before the punctuations for better tokenization
    '''
    filters = ["!", "#", "$", "%", "(", ")", "/", "*", ":", ";", "<", "=", ">", "?", "@", "[",
               "\\", "]", "_", "`", "{", "}", "~", ",","."]
    text = text.strip('\n ')
    text = text.replace('\n', ' ').replace('\r', ' ').replace('\br', ' ')
    for i in text:
        i = i.replace("'", ' ').replace("`", ' ').replace("°",'')
        if i in filters:
            text = text.replace(i, " "+i+' ')
    text = " ".join(text.split())
    return text

def create_data(df, text, filepath):
    '''
    The function responsible for the creation of data in the said format.
    1234
    1>1,a>
    '''
    appended = r'C:/Data/training/train.txt'
    with open(filepath , 'a', newline='', encoding="utf-8") as f:
        for sentence in text:
            text_ = sentence       
            match_list = []
            for i,y in df:
                a, text_ = matcher(str(sentence), i)
                if a:
                    match_list.append((a[0][0], a[0][1], y))
            print(f'FINAL KEYS : : : {match_list}')
            if match_list:
                d = sen2tag(str(sentence), match_list)#mark_sentence(str(sentence), match_list)
                for i in d:
                    f.writelines(str(i)+'\n')
                f.writelines('\n')

In [46]:
def main():
    
    names = ['test','MSEDCL','BHEL','RGGVY','ONGC','ADANI' ,'AGRA' ,'BSES', 'DUGJY' ,'KEONJ' ,'Reliance' ,'TATA' ,'TSLT', 'APDCL', 'KVX', 'PVVNL',
            'ADANILT','APGENCO','BESCOM','BESCOMCC','BESCOMHT','BMRCL','Cables 02','CIDCO','CSL','DGVCL','IPDS','JBVNL','NESCO','NRDAHT',
            'NRDALT','PGVCL','PSPCL','PVCPOWER','TNEB','TNEB2','TNEBLV','TNEBXLPE','TSECL',
            '44AD', 'AIIMS', 'APDCL-AIIB', 'AVVNL', 'BEL', 'BWE', 'CABLE-AIIMS', 'Cables_Spec', 'CESU MVCC', 'DDUGJY Control Cable', 'DHBVN', 'DHBVN CSC-65 LT UA', 'DHBVN CSC-67 AB', 'DHBVN' ,'DHBVN_CSC', 'EHV' ,'GED' ,'GETCO LT', 'GETCO' ,'GETCO2',
            'GMRC', 'HTABMGVCL', 'HTCable', 'HT_GEB', 'IOCL MEERUT' ,'IOCL' ,'IOCL_HV', 'IOCL_PARADIP', 'ITD_JNPT' ,'JACOB', 'Jajpur' ,'JSBAY', 'KMRCL 33KV' ,'KMRCL', 'KPTCL', 'KSEB' ,'LVAB_CABLE', 'LV_ALU' ,'MECON' ,'NMDC',
            'NRDA' ,'RANCHI' ,'RRCAT' ,'RSP', 'SRF' ,'T S_CABLE-AIIMS' ,'UPMRC' ,'XLPE_CABLE','DHBVN_111']
    
    for doc in tqdm(names,ncols=150,
                desc=f" Processing Files. . . . "):
        try:
            print(f'File currently being worked on: {doc}')
            filename = doc 
            #path to save the txt file.
            pdf = r'C:/Data/raw/reworked/'+filename +'.pdf'
            filepath = r'C:/Data/raw/reworked/'+filename +'.txt'
            sentoken = r'C:/Data/training/'+filename +'.csv'
            final = r'C:/Data/train.txt'
            
            ## creating the file.
            sentences = []
            pdf2token(pdf,sentoken)
            data = txttrain(filepath)    
            
            with open(sentoken, 'r', encoding="utf-8") as read_obj:
                csv_reader = csv.reader(read_obj)
                for row in csv_reader:
                    sentences.append(row)
            create_data(data, sentences, final)
        except FileNotFoundError:
            pass  

if __name__ == '__main__':
    main()


 Processing Files. . . . :   0%|                                                                                               | 0/88 [00:00<?, ?it/s]

File currently being worked on: test
File currently being worked on: MSEDCL
File currently being worked on: BHEL
File currently being worked on: RGGVY
File currently being worked on: ONGC
File currently being worked on: ADANI
File currently being worked on: AGRA
File currently being worked on: BSES
File currently being worked on: DUGJY
File currently being worked on: KEONJ
File currently being worked on: Reliance
File currently being worked on: TATA
File currently being worked on: TSLT
File currently being worked on: APDCL
File currently being worked on: KVX
File currently being worked on: PVVNL
File currently being worked on: ADANILT
File currently being worked on: APGENCO
File currently being worked on: BESCOM
File currently being worked on: BESCOMCC
File currently being worked on: BESCOMHT
File currently being worked on: BMRCL
File currently being worked on: Cables 02
File currently being worked on: CIDCO
File currently being worked on: CSL
File currently being worked on: DGVCL
File

 Processing Files. . . . :  45%|███████████████████████████████████████                                               | 40/88 [00:01<00:01, 38.16it/s]

FINAL KEYS : : : [(356, 363, 'standard')]
>> 356 363 standard First: 0 - 356
'XXXXXXX : cross - linked polyethylene insulated pvc sheathed cables for ( part 2 ) working voltages from 3.3 kv up to and including 33 kv is 8130 : conductors for insulated electric cables and flexible cords is 5831 : pvc insulation and sheath of electric cables is 3975 : mild steel wires , formed wires and tapes for armouring of cables is 4826 : hot dip galvanized coatings on round steel wires is 2633 : methods for testing uniformity of coating of zinc coated articles is 3961 : recommended current ratings for cables pvc insulated and pvc ( part 2 ) sheathed heavy duty cables is 3961 : recommended current ratings for cables xlpe insulated and pvc ( part 6 ) sheathed cables ( part 7 ) thermoplastic sheathed cables is 613 : copper rods and bars for electrical purpose .' //||// is 7098
'is 7098 : cross - linked polyethylene insulated pvc sheathed cables for ( part 2 ) working voltages from 3.3 kv up to and inclu

 Processing Files. . . . :  45%|███████████████████████████████████████                                               | 40/88 [00:12<00:01, 38.16it/s]

...

File currently being worked on: XLPE_CABLE
2022-02-08 13:26:58,112 Warning: An empty Sentence was created! Are there empty strings in your dataset?
[ 11kv xlpe cable , cableItype ]
[ high conductivity stranded high density aluminum , conductor ]
[ compacted circular , stranded , conductor ]
[ semiconducting polymer shall be cross linked , conductorIscreen ]
[ xlpe ( cross linked poly ethylene , insulation ]
[ non-metallic semi-conducting shield , insulationIscreen ]
[ copper metallic overlapped tape shield , insulationIscreen ]
[ pvc fillers , filler ]
[ non hygroscopic pvc , filler ]
[ pvc sheath shall be extruded , innerIsheath ]
[ galvanized steel strip armoring , armour ]
[ extruded pvc outer sheath , outerIsheath ]
[ frlsh , ( flame retardant low smoke , outerIsheath ]
[ wooden or steel drums of heavy construction , packing ]
[ sound and free from defects , packing ]
[ wood preservative shall be applied to the entire drum , packing ]
[ name of manufacturer , marking ]
[ year of ma

 Processing Files. . . . :  99%|█████████████████████████████████████████████████████████████████████████████████████ | 87/88 [02:32<00:01,  1.74s/it]


FINAL KEYS : : : []
FINAL KEYS : : : []
'further , these specifications are subject to the instructions to tenderers , general technical specifications , terms and conditions mentioned in general requirement of specifications , and XXXX form b. in case if any ambiguity of technical details given elsewhere the conditions given in technical specifications shall prevail .' //||// upcl
FINAL KEYS : : : [(192, 196, 'marking')]
>> 192 196 marking First: 0 - 192
'guaranteed technical particulars for XXXXXXXXXXXXXXX of size ( i ) 3x50sq.mm ( ii ) 3x70sq.mm , ( iii ) 3x120sq.mm , ( iv ) 3x185sq.mm ( v ) 3x300sq.mm & ( vi ) 1x630sq.mm sl.n o .' //||// 11kv xlpe cable
FINAL KEYS : : : [(38, 53, 'cableItype')]
>> 38 53 cableItype First: 0 - 38
'particulars unit XXXXXXXXXXXXXXX name of manufacturer cable sizes' //||// 11kv xlpe cable
'particulars unit 11kv xlpe cable XXXXXXXXXXXXXXXXXXXX cable sizes' //||// name of manufacturer
FINAL KEYS : : : [(18, 33, 'cableItype'), (34, 54, 'marking')]
>> 18 3

 Processing Files. . . . : 100%|██████████████████████████████████████████████████████████████████████████████████████| 88/88 [02:34<00:00,  1.76s/it]

FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
FINAL KEYS : : : []
'ka 18 voltage drop per thousand m length at rated current a ) when laid directly in ground volt / km b ) when laid directly in covered trench volt / km c ) when laid directly in air volt / km 19 impulse voltage withstand kv 20 derating factors under various conditions of installation o c ( max . ) a ) d.c resistance per core at 20 o ohm / km c ( max . ) b ) ac impedance per core at 20 c ) ac reactance per core at 20oc ( max. ) ohm / km d ) capacitance per